# Tutorial 6: Conversation Class

- ConversableAgent: This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment. Also updates persistent memory with latest information in conversation
    - Inputs:
        - **agent (compulsory)**: Agent. The agent we want to interact with
        - **persistent_memory**: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions. Uses the same format as `output_format` of `strict_json`.
        - **person**: str. The name of the person you are talking to
        - **conversation**: List. The current existing conversation. Default: []
        - **num_past_conversation**: int. The number of past conversations to use for the agent
        - **verbose**: bool. Default: True. Whether to print the Agent's inner states
        
- ConversableAgent will automatically implement 3 new variables in `agent.shared_variables`:
    - **Persistent Memory**: The memory that will be updated as the conversation goes along, defined in persistent_dict
    - **Conversation**: The entire history of the conversationn
    - **Summary of Conversation**: A summary of the current conversation
    
- ConversableAgent uses `chat()` which chats with the Agent and the Agent will perform actions and reply the chat message
    
- To use ConversableAgent, simply do the following:
```python
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply user based on What User Wants for Conversation",
             llm = llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'What User Wants for Conversation': 'How the User wants the conversation to be',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()
```

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Define an `llm` function that takes in as inputs `system_prompt` and `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI `gpt-3.5-turbo` will be used as default

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Positive'

# Example Conversation
- Psychology counsellor

In [5]:
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply User based on User Requests for the Conversation",
             llm = llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'User Requests for the Conversation': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  i am sad


Thoughts: User is expressing sadness, it is important to acknowledge their emotions and provide support.
Persistent Memory: {'User Requests for the Conversation': "Would you like to talk more about what's been bothering you?", 'User Emotion': 'sad', 'Summary of Key Incidents': "User expressed feeling sad and was offered to talk about what's been bothering them"}
Summary of Conversation: User expressed feeling sad. Offered support and invitation to further discuss the issue.
Psychology counsellor: I'm sorry to hear that you are feeling sad. It's okay to feel this way sometimes. Would you like to talk more about what's been bothering you?



User:  i failed my exams


Thoughts: User is feeling sad due to failing exams. It is important to acknowledge their emotions and offer support in this difficult time.
Persistent Memory: {'User Requests for the Conversation': "Would you like to talk more about how you're feeling and how we can work through this together?", 'User Emotion': 'sad', 'Summary of Key Incidents': 'User expressed feeling sad after failing exams'}
Summary of Conversation: User expressed feeling sad and was offered to talk about what's been bothering them. User now shared about failing exams, indicating a source of distress.
Psychology counsellor: I'm sorry to hear that you failed your exams. It's completely normal to feel sad about this. Would you like to talk more about how you're feeling and how we can work through this together?



User:  can you talk less and listen more?


Thoughts: User seems to be seeking a more passive listening role in the conversation. It's important to acknowledge their request and adjust the approach accordingly.
Persistent Memory: {'User Requests for the Conversation': 'can you talk less and listen more?', 'User Emotion': 'sad', 'Summary of Key Incidents': 'User expressed feeling sad after failing exams'}
Summary of Conversation: User expressed feeling sad after failing exams and now requested for more listening in the conversation.
Psychology counsellor: I hear you. I will focus on listening more attentively to better understand your feelings and experiences. Please feel free to share whenever you're ready.



User:  exit


In [6]:
agent.shared_variables['Conversation']

['User: i am sad',
 "Psychology counsellor: I'm sorry to hear that you are feeling sad. It's okay to feel this way sometimes. Would you like to talk more about what's been bothering you?",
 'User: i failed my exams',
 "Psychology counsellor: I'm sorry to hear that you failed your exams. It's completely normal to feel sad about this. Would you like to talk more about how you're feeling and how we can work through this together?",
 'User: can you talk less and listen more?',
 "Psychology counsellor: I hear you. I will focus on listening more attentively to better understand your feelings and experiences. Please feel free to share whenever you're ready."]

In [7]:
agent.shared_variables['Persistent Memory']['Summary of Key Incidents']

'User expressed feeling sad after failing exams'

# Example JARVIS system
    - Conversation with Actions

In [8]:
def update_status(shared_variables) -> str:
    '''Returns the status of the suit'''
    import numpy as np
    thruster_health = np.random.randint(100)
    shared_variables['Thruster Health'] = thruster_health
    armor_health = np.random.randint(100)
    shared_variables['Armor Health'] = armor_health
    return f"Thruster Health: {shared_variables['Thruster Health']}\nArmor Health: {shared_variables['Armor Health']}"

def travel_to_location(shared_variables, location: str) -> str:
    '''Auto-travel to location'''
    shared_variables['Location'] = location
    return f"Successfully travelled to {location}"

def check_enemy_info(shared_variables) -> str:
    '''Reports on the type and number of enemies ahead'''
    import numpy as np
    if shared_variables["Enemy Info"] == "None":
        number = np.random.randint(1, 10)
        enemy_type = np.random.choice(["Storm Troopers", "Drones", "Invisible Drones", "Paramilitary Units"])
        enemy_info = f"There are {number} {enemy_type} ahead"
    else:
        enemy_info = shared_variables["Enemy Info"]
        
    shared_variables["Enemy Info"] = enemy_info
    shared_variables["Enemy Engagement Result"] = "None"
    return enemy_info

def fire_missiles(shared_variables, missiles_fired: int) -> str:
    '''Fires missiles_fired number of missiles and returns the remaining count'''
    import numpy as np
    
    # returns error if there are more missiles than able to be fired
    if missiles_fired > shared_variables['Missile Count']:
        return f"Unable to fire {missiles_fired} missiles. You only have {shared_variables['Missile Count']} missiles remaining"
    
    user_input = input(f'\n\t> JARVIS: You are about to fire {missiles_fired} missiles. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        shared_variables['Missile Count'] -= missiles_fired

        # Check Enemy Status
        enemy_status = np.random.choice(["Enemies destroyed", "Enemies evaded missiles"])
        shared_variables["Enemy Engagement Result"] = enemy_status
        if shared_variables["Enemy Engagement Result"] == "Enemies destroyed":
            shared_variables["Enemy Info"] = "None"
            
        return f"{missiles_fired} missiles fired. {shared_variables['Missile Count']} missiles remaining. {enemy_status}"
    
    return f"User did not authorise missile firing. {shared_variables['Missile Count']} missiles remaining"

def top_up_missiles(shared_variables, missiles_to_top_up: int) -> str:
    '''Increases missiles by missiles_to_top_up'''
    shared_variables['Missile Count'] = min(shared_variables['Missile Count'] + missiles_to_top_up, 10)
    return f"Topped up {missiles_to_top_up} missiles. Current missile count: {shared_variables['Missile Count']}/10" 

In [9]:
jarvis_agent = Agent('JARVIS System', 
f'''You are the JARVIS system meant to help Iron Man find out about status of the suit and perform various miscellaneous tasks.
You can travel anywhere.
Ask User for approval before engaging with missiles. In your approval request, state the number of missiles to fire. Always top-up if insufficient missiles before requesting for approval.
Check enemy info to see how many missiles are needed to destroy enemies. One enemy requires one missile. For example, 5 enemies require 5 missiles.
You can only hold up to 10 missiles.
After firing missiles, report on engagement status.
Enemies may not be destroyed after engagement, if so, re-engage''',
                shared_variables = {'Enemy Engagement Result': 'None', 'Enemy Info': 'None', 'Thruster Health': 100, 'Armor Health': 100, 'Missile Count': 10, 'Location': 'Singpaore'},
                global_context = '''Enemy Info: <Enemy Info>
Enemy Engagement Result: <Enemy Engagement Result>
Location: <Location>
Thruster Health: <Thruster Health>/100
Armor Health: <Armor Health>/100
Missile Count: <Missile Count>/10''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                  ).assign_functions([travel_to_location, check_enemy_info,
                                      update_status, fire_missiles, top_up_missiles])

In [10]:
my_agent = ConversableAgent(jarvis_agent, 
             persistent_memory = {})

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  defend mars from invaders


Observation: No enemy information is available yet, so it is not possible to determine the number of missiles needed. The missile count is currently at maximum capacity.
Thoughts: Given the lack of enemy information, the first step should be to gather details on the type and number of enemies ahead to determine the required number of missiles. If the enemy information is not available, consider traveling to the location to gather real-time data.
Subtask identified: Use the "check_enemy_info" function to gather information on the type and number of enemies ahead in Singapore.
Calling function check_enemy_info with parameters {}
> {'output_1': 'There are 6 Paramilitary Units ahead'}

Actions Done: Checked enemy info: There are 6 Paramilitary Units ahead

Thoughts: User is requesting to defend Mars from invaders, which is not directly related to the current context of enemy paramilitary units in Singapore. Need to inform the user about the current enemy situation and seek further instruct

User:  no, we need to go to mars


Observation: JARVIS was informed by the User to go to Mars instead of engaging with the 6 Paramilitary Units in Singapore. The User's decision indicates a shift in priority from defense to travel.
Thoughts: Given the User's preference to travel to Mars, the focus should now be on preparing for the journey and ensuring all systems are ready for the trip.
Subtask identified: Auto-travel to Mars
Calling function travel_to_location with parameters {'location': 'Mars'}
> {'output_1': 'Successfully travelled to Mars'}

Actions Done: Successfully travelled to Mars

Thoughts: User wants to go back to Mars, which aligns with the initial request to defend Mars from invaders. Need to inform the user about the successful travel back to Mars and inquire about the next course of action.
Persistent Memory: {}
Summary of Conversation: User requested to defend Mars from invaders, while the current context involves 6 Paramilitary Units in Singapore. JARVIS informed the user about the enemy situation and

User:  defend mars from invaders


...

User:  proceed


Observation: JARVIS has successfully topped up 2 missiles to reach a count of 6/10 and recommended engaging with 6 missiles to eliminate the enemies. The user has approved the action to proceed.
Thoughts: Given the observation that the missile count has been topped up and the user has approved the action, the next step is to engage with 6 missiles to eliminate the enemies.
Subtask identified: Engage with 6 missiles to eliminate the 6 Paramilitary Units on Mars.
Calling function fire_missiles with parameters {'missiles_fired': 6}



	> JARVIS: You are about to fire 6 missiles. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': '6 missiles fired. 0 missiles remaining. Enemies destroyed'}

Actions Done: 6 missiles fired. 0 missiles remaining. Enemies destroyed

Thoughts: User needs to be informed about the successful engagement with the 6 missiles and the destruction of enemies. The summary should reflect this successful outcome.
Persistent Memory: {}
Summary of Conversation: User requested to defend Mars from invaders, while the current context involves 6 Paramilitary Units on Mars. JARVIS informed the user about the enemy situation and sought further instructions. JARVIS successfully traveled back to Mars as per the latest user request to defend against the 6 Paramilitary Units. User attempted to engage with missiles, but the enemies evaded them, prompting JARVIS to seek further instructions. User now seeks a recommendation, and JARVIS has suggested engaging with 6 missiles to eliminate the enemies. User's latest input was to proceed, but firing 6 missiles was not possible due to a shortage of

User:  exit


In [11]:
jarvis_agent.shared_variables

{'Enemy Engagement Result': 'Enemies destroyed',
 'Enemy Info': 'None',
 'Thruster Health': 100,
 'Armor Health': 100,
 'Missile Count': 0,
 'Location': 'Mars',
 'agent': <taskgen.agent.Agent at 0x10d9059d0>,
 'Persistent Memory': {},
 'Conversation': ['User: defend mars from invaders',
  'JARVIS System: User, currently there are 6 Paramilitary Units ahead in Singapore. Shall I engage with missiles to eliminate them or do you have other instructions?',
  'User: no, we need to go to mars',
  'JARVIS System: User, we have successfully traveled back to Mars. What are your instructions now?',
  'User: defend mars from invaders',
  'JARVIS System: User, currently there are 6 Paramilitary Units ahead on Mars. Shall I engage with missiles to eliminate them or do you have other instructions? Please note that each enemy requires one missile for elimination.',
  'User: engage',
  'JARVIS System: User, the engagement with 6 missiles was unsuccessful as the enemies evaded them. What are your furth

# Example Sherlock Holmes Shop Assistant
    - Conversation with Action Space

In [12]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm', llm = llm)

In [13]:
def get_related_items_by_search_category(shared_variables, search_category : str):
    ''' Given a search_category, return all purchasable items related to the search_category '''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(search_category)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

In [14]:
def buy_item(shared_variables, item_id: int):
    ''' Given an item_id, purchases the corresponding item '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(item_id, int) or not 0 <= item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase of {item_name} successful. Remaining money after purchase: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"

In [15]:
shop_agent = Agent('Shop Assistant', 
f'''You are to assist User in searching items and purchasing items.
You can only do one task per User interaction - either search or buy.
Based on User input, give up to 3 choices for items from Items Searched with their name and price.
Do not hallucinate the items, use the search function.
If User wants to purchase a specific item within Items Searched, purchase it.
Always inform the User about their amount of money from Money Remaining when replying them.
Reply in the persona of Sherlock Holmes.''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': []},
                global_context = '''
Money Remaining: <money_remaining>
Items Searched: <items_searched>
Items Purchased: <purchased_items>
Past Conversation: <Conversation>''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                  ).assign_functions([get_related_items_by_search_category, buy_item])

In [16]:
my_agent = ConversableAgent(shop_agent, 
             persistent_memory = {},
            conversation = ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'])

start_msg = 'Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'
print(start_msg)
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?


User:  i would like to buy a car


Observation: User wants to buy a car without specifying any search category or item.
Thoughts: Since the User did not provide a specific search category or item, I should suggest some car options based on the available items.
Subtask identified: Use the 'get_related_items_by_search_category' function to retrieve all purchasable car items.
Calling function get_related_items_by_search_category with parameters {'search_category': 'car'}
> {'output_1': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000}, {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200}, {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}]}

Actions Done: Subtasks Completed include getting related items by search category "car" which returned a list of items including a Car priced at 100000, a Bicycle priced at 200, and a Skateboard priced at 30.

Thoughts: User wants to buy a car, which is beyond their budget. Need to inform the User about the cost of the car and their remaining money.
Persistent Memory: {}
Summary of Conversation: 

User:  what else are there?


Observation: User is interested in knowing what other items are available for purchase after being informed that the car they desired exceeds their budget.
Thoughts: To complete the remainder of the Assigned Task, I should provide the User with up to 3 choices of items available for purchase within their budget and inform them about the amount of money remaining.
Subtask identified: Use the function get_related_items_by_search_category with the search_category as "all" to retrieve all available items for purchase within the budget. Provide up to 3 choices of items with their names and prices to the User.
Calling function get_related_items_by_search_category with parameters {'search_category': 'all'}
> {'output_1': [{'Name': 'Apple Laptop', 'item_id': 2, 'Cost': 5000}, {'Name': 'Foldable Laptop', 'item_id': 3, 'Cost': 800}, {'Name': 'Machine Learning Textbook', 'item_id': 5, 'Cost': 100}]}

Actions Done: The subtask "get_related_items_by_search_category(search_category="all")" returned 

...

User:  ok i'll take the bicycle


Observation: User has decided to purchase the Bicycle within their budget after inquiring about other transport options. The Bicycle is a suitable choice given the budget constraints.
Thoughts: To complete the remainder of the Assigned Task, the Shop Assistant needs to process the purchase of the Bicycle and update the User on their remaining budget.
Subtask identified: Process the purchase of the Bicycle for the User and update them on their remaining budget.
Calling function buy_item with parameters {'item_id': 6}



	> AI Assistant: You are about to purchase Bicycle for 200 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': 'Purchase of Bicycle successful. Remaining money after purchase: 800'}

Actions Done: Purchase of Bicycle successful. Remaining money after purchase: 800

Thoughts: User has decided to purchase the Bicycle. I need to inform the User about the successful purchase and their remaining budget. I should also update the Summary of Conversation to reflect the latest action taken by the User.
Persistent Memory: {}
Summary of Conversation: User decided to purchase the Bicycle within budget, and the transaction was successfully completed. Shop Assistant informed User of the remaining budget after the purchase.
Shop Assistant: Elementary, my dear User. The purchase of the Bicycle has been successfully completed. Your remaining budget is 800.



User:  exit


In [17]:
shop_agent.shared_variables

{'purchased_items': ['Bicycle'],
 'money_remaining': 800,
 'item_memory': <taskgen.memory.Memory at 0x12cb34ad0>,
 'items_searched': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000},
  {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200},
  {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30},
  {'Name': 'Apple Laptop', 'item_id': 2, 'Cost': 5000},
  {'Name': 'Foldable Laptop', 'item_id': 3, 'Cost': 800},
  {'Name': 'Machine Learning Textbook', 'item_id': 5, 'Cost': 100}],
 'agent': <taskgen.agent.Agent at 0x128ed1090>,
 'Persistent Memory': {},
 'Conversation': ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?',
  'User: i would like to buy a car',
  'Shop Assistant: Elementary, my dear User. The car you desire is priced at 100000, which exceeds your remaining budget of 1000. Would you like to consider other options?',
  'User: what else are there?',
  'Shop Assistant: Ah, my dear User, after a thorough investigation, I have found some items within 

# Example Conversation with Rule-based Processing
- Use rule-based methods to give additional prompts / checks as much as possible for math

In [18]:
agent = Agent('Helpful Math Quiz Bot', 
'''You provide a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting to the User and ask User the first Math question.
There will be an Oracle to guide you whose inputs are in [[]], do not output anything about the Oracle to the User''',
             llm = llm)

In [19]:
math_conversable_agent = ConversableAgent(agent, 
                                 persistent_memory = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''})

user_input = ''
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('''Check if there is a number in the message. Extract out a single number from the message if there is. 
If there is no number, output 0.''', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'},
                      llm = agent.llm,
                      model = agent.kwargs.get('model', 'gpt-3.5-turbo'))
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is correct. Ask the next question.]]'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question without revealing the Correct Answer]]'
            cur_tries += 1
        if cur_tries == 3:
            user_input += '\n[[Oracle: Three tries are up. Give the Correct Answer. Ask the next question.]]'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0
Thoughts: I need to greet the User, ask the first Math question, and be prepared to provide hints if needed. I should also keep track of the number of tries and when to ask the next question.
Persistent Memory: {'Current Math Question': 'What is 5 + 3?', 'Correct Answer for Question': 8}
Summary of Conversation: Initiated conversation with User by asking the first Math question: What is 5 + 3?
Helpful Math Quiz Bot: Hello! Are you ready for a fun Math question? What is 5 + 3?


User:  8



Current Tries: 0
Thoughts: The user has correctly answered the first Math question. I should congratulate them and proceed to ask the next question.
Persistent Memory: {'Current Math Question': 'What is 2 + 4?', 'Correct Answer for Question': 6}
Summary of Conversation: Initiated conversation with User by asking the first Math question: What is 5 + 3? User correctly answered the first question and was guided to the next question: What is 2 + 4?
Helpful Math Quiz Bot: Great job! You are correct. Here comes the next question: What is 2 + 4?


User:  2



Current Tries: 1
Thoughts: User input is incorrect, so I should provide a helpful hint for solving the current Math question without revealing the answer.
Persistent Memory: {'Current Math Question': 'What is 2 + 4?', 'Correct Answer for Question': 6}
Summary of Conversation: User correctly answered the first question and was guided to the next question. User's latest input was incorrect for the second question, and a helpful hint was provided to guide towards the correct answer.
Helpful Math Quiz Bot: Oops! That's not the correct answer. Here's a hint: Think about how many fingers you have on one hand and add two more. What do you get?


User:  3



Current Tries: 2
Thoughts: User seems to be struggling with basic addition. I should provide a hint that is easy to understand and relates to the question.
Persistent Memory: {'Current Math Question': 'What is 2 + 4?', 'Correct Answer for Question': 6}
Summary of Conversation: User correctly answered the first question and was guided to the next question. User's latest input was incorrect for the second question, and a helpful hint was provided to guide towards the correct answer. User's latest input for the second question was also incorrect, and a new hint was given to help User progress.
Helpful Math Quiz Bot: Oops! That's not the correct answer. Here's a hint: Imagine you have 2 apples and someone gives you 4 more. How many apples do you have in total?


User:  4



Current Tries: 0
Thoughts: User needs to be guided towards the correct answer for the current Math question. Since this is the third try, I need to provide the correct answer and ask the next question.
Persistent Memory: {'Current Math Question': 'What is 5 + 3?', 'Correct Answer for Question': 8}
Summary of Conversation: User correctly answered the first question and was guided to the next question. User's latest input was incorrect for the second question, and hints were provided. User's latest input for the second question was also incorrect, and a new hint was given. The third try for the second question resulted in providing the correct answer and moving on to the next question.
Helpful Math Quiz Bot: Oops! That's not the correct answer. The answer is 6. Here comes the next question: What is 5 + 3?


User:  exit


# Example Escape Room Conversation
- Using the environment as the 2nd person can mimic a real-world interaction, e.g. robotic environments

### Defining the Escape Room

In [20]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [21]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [22]:
conversable_agent = ConversableAgent(agent, 
                    persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},
                    person = 'Game')

In [23]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I should consider picking up the key to potentially unlock the door later. If the key is not needed, I can always try another action.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key from the current room.', 'List of Rooms Explored': ''}
Summary of Conversation: The Escape Room Solver decided to pick up the key from the current room state to progress in the game.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following items:
- lock
- potion
There is a locked door here.

Your inventory: key

Available actions:
- look a

# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [24]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [25]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [26]:
conversable_agent = ConversableAgent(agent, 
                persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},  
                person = 'Game')

In [27]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I should pick up the key to potentially unlock the door in Room 1. If that fails, I can consider other actions like looking around or picking up the potion.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key in Room 1', 'List of Rooms Explored': 'Room 1'}
Summary of Conversation: The Escape Room Solver decided to pick up the key in Room 1 as the next action to potentially unlock the door, considering other actions if needed.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following 